# 🚀 Training Model trên Google Colab

Notebook này giúp bạn train model trên Google Colab với GPU miễn phí.

## 📋 Hướng dẫn:
1. Chạy các cell theo thứ tự
2. Upload dữ liệu hoặc sử dụng Google Drive
3. Chọn GPU runtime: Runtime → Change runtime type → GPU
4. Chạy training và lưu kết quả về Google Drive


## 1. Cài đặt môi trường và dependencies


In [ ]:
# Cài đặt các thư viện cần thiết
%pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install -q albumentations tqdm tensorboard onnx onnxruntime
%pip install -q pandas numpy Pillow opencv-python
%pip install -q kagglehub

print("✅ Đã cài đặt xong các thư viện!")


## 2. Kiểm tra GPU


In [ ]:
import torch
import torch.cuda as cuda

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {cuda.is_available()}")
if cuda.is_available():
    print(f"CUDA version: {cuda.version()}")
    print(f"GPU device: {cuda.get_device_name(0)}")
    print(f"GPU memory: {cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️  Không có GPU! Vui lòng chọn GPU runtime:")
    print("   Runtime → Change runtime type → Hardware accelerator → GPU")


## 3. Mount Google Drive (để lưu kết quả)


In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Tạo thư mục lưu kết quả trên Drive
DRIVE_RESULTS_DIR = '/content/drive/MyDrive/age_gender_emotion_training'
os.makedirs(DRIVE_RESULTS_DIR, exist_ok=True)
print(f"✅ Đã mount Google Drive")
print(f"📁 Kết quả sẽ lưu tại: {DRIVE_RESULTS_DIR}")


## 4. Upload code từ máy tính

**Cách 1: Upload trực tiếp (khuyến nghị)**
- Click vào icon folder bên trái
- Upload thư mục `training_experiments` vào `/content/`
- Hoặc upload file zip và giải nén

**Cách 2: Clone từ GitHub (nếu code đã có trên GitHub)**


In [ ]:
# Tạo thư mục project
!mkdir -p /content/project
%cd /content/project

# Option 1: Clone từ GitHub (nếu có)
# !git clone https://github.com/your-username/your-repo.git .
# %cd training_experiments

# Option 2: Upload code từ máy tính
# Sử dụng file uploader ở cell tiếp theo hoặc upload thủ công qua file browser

print("📁 Đã tạo thư mục project tại /content/project")
print("\n📤 Bây giờ hãy upload code:")
print("   1. Click vào icon folder bên trái (📁)")
print("   2. Upload thư mục training_experiments vào /content/project/")
print("   3. Hoặc upload file zip và giải nén ở cell tiếp theo")


In [ ]:
# Upload và giải nén code (nếu upload dạng zip)
from google.colab import files
import zipfile
import os

print("📤 Upload file zip chứa code training_experiments.zip")
print("   (Bỏ qua nếu đã upload thư mục trực tiếp)")

uploaded = files.upload()

# Giải nén nếu có file zip
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"\n📦 Đang giải nén {filename}...")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('/content/project')
        print(f"✅ Đã giải nén vào /content/project/")
        os.remove(filename)  # Xóa file zip sau khi giải nén

# Kiểm tra cấu trúc
import os
from pathlib import Path

project_dir = Path('/content/project/training_experiments')
if project_dir.exists():
    print(f"\n✅ Đã tìm thấy training_experiments tại: {project_dir}")
    print(f"   Các file quan trọng:")
    files_to_check = [
        'train_week2_lightweight.py',
        'src/data/dataset.py',
        'src/models/mobileone.py'
    ]
    for f in files_to_check:
        file_path = project_dir / f
        print(f"   {'✅' if file_path.exists() else '❌'} {f}")
else:
    print(f"\n⚠️  Chưa tìm thấy training_experiments!")
    print(f"   Hãy upload code vào /content/project/")


## 5. Upload dữ liệu training

**Có 3 cách để có dữ liệu:**

1. **Upload từ máy tính** (cho dataset nhỏ < 2GB)
2. **Download từ Kaggle** (khuyến nghị cho dataset lớn)
3. **Sử dụng từ Google Drive** (nếu đã upload trước đó)


### Option A: Upload từ máy tính (cho dataset nhỏ)


In [ ]:
# Upload file zip chứa dữ liệu đã processed
from google.colab import files
import zipfile
from pathlib import Path

print("📤 Hãy upload file zip chứa dữ liệu (data_processed.zip)")
print("   File zip nên có cấu trúc: processed/train/, processed/val/, processed/test/")

uploaded = files.upload()

# Giải nén
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"\n📦 Đang giải nén {filename}...")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('/content/project/training_experiments/data')
        print(f"✅ Đã giải nén vào /content/project/training_experiments/data/")
        os.remove(filename)  # Xóa file zip sau khi giải nén


### Option B: Download dataset từ Kaggle (khuyến nghị)


In [ ]:
# Cài đặt kaggle API
%pip install -q kaggle

# Upload kaggle.json từ máy tính
# Lấy từ: https://www.kaggle.com/settings -> API -> Create New Token
print("📤 Upload file kaggle.json (từ ~/.kaggle/kaggle.json)")
print("   Bỏ qua nếu đã upload trước đó")
from google.colab import files
uploaded = files.upload()

# Di chuyển kaggle.json vào đúng vị trí
if 'kaggle.json' in uploaded:
    import os
    os.makedirs(os.path.expanduser('~/.kaggle'), exist_ok=True)
    import shutil
    shutil.move('kaggle.json', os.path.expanduser('~/.kaggle/kaggle.json'))
    os.chmod(os.path.expanduser('~/.kaggle/kaggle.json'), 0o600)
    print("✅ Đã setup Kaggle API")

# Download datasets
import kagglehub
from pathlib import Path

DATA_DIR = Path('/content/project/training_experiments/data')
DATA_DIR.mkdir(parents=True, exist_ok=True)

print("\n📥 Đang download datasets...")
print("   (Có thể mất vài phút)")

# Download UTKFace
try:
    utkface_path = kagglehub.dataset_download("jangedoo/utkface-new")
    print(f"✅ UTKFace: {utkface_path}")
except Exception as e:
    print(f"⚠️  Lỗi download UTKFace: {e}")

# Download FER2013
try:
    fer2013_path = kagglehub.dataset_download("msambare/fer2013")
    print(f"✅ FER2013: {fer2013_path}")
except Exception as e:
    print(f"⚠️  Lỗi download FER2013: {e}")

print("\n✅ Hoàn tất download datasets")
print("\n⚠️  Lưu ý: Cần chạy script preprocess để tạo train/val/test splits")


### Option C: Sử dụng dữ liệu từ Google Drive


In [ ]:
# Copy dữ liệu từ Google Drive
import shutil
import os
from pathlib import Path

DRIVE_DATA_DIR = '/content/drive/MyDrive/age_gender_emotion_data'
LOCAL_DATA_DIR = Path('/content/project/training_experiments/data/processed')

if os.path.exists(DRIVE_DATA_DIR):
    print(f"📁 Copy dữ liệu từ {DRIVE_DATA_DIR}...")
    shutil.copytree(DRIVE_DATA_DIR, LOCAL_DATA_DIR, dirs_exist_ok=True)
    print(f"✅ Đã copy dữ liệu vào {LOCAL_DATA_DIR}")
else:
    print(f"⚠️  Không tìm thấy dữ liệu tại {DRIVE_DATA_DIR}")
    print("   Hãy upload dữ liệu vào Google Drive trước")


## 6. Kiểm tra và chuẩn bị dữ liệu


In [ ]:
# Kiểm tra dữ liệu đã sẵn sàng chưa
from pathlib import Path

data_dir = Path('/content/project/training_experiments/data/processed')

train_dir = data_dir / 'train'
val_dir = data_dir / 'val'
test_dir = data_dir / 'test'

print("📊 Kiểm tra dữ liệu:")
print(f"   Train: {train_dir.exists()}")
print(f"   Val: {val_dir.exists()}")
print(f"   Test: {test_dir.exists()}")

if train_dir.exists():
    # Đếm số ảnh
    train_images = list(train_dir.glob('**/*.jpg')) + list(train_dir.glob('**/*.png'))
    val_images = list(val_dir.glob('**/*.jpg')) + list(val_dir.glob('**/*.png')) if val_dir.exists() else []
    test_images = list(test_dir.glob('**/*.jpg')) + list(test_dir.glob('**/*.png')) if test_dir.exists() else []
    
    print(f"\n📸 Số lượng ảnh:")
    print(f"   Train: {len(train_images)}")
    print(f"   Val: {len(val_images)}")
    print(f"   Test: {len(test_images)}")
    
    if len(train_images) == 0:
        print("\n⚠️  Chưa có ảnh trong thư mục train!")
        print("   Cần chạy script preprocess trước")
else:
    print("\n⚠️  Chưa có dữ liệu processed!")
    print("   Cần download và preprocess dữ liệu trước")


## 7. Chạy Training

### Training với cấu hình mặc định


In [ ]:
import sys
import os
from pathlib import Path

# Thêm project vào Python path
project_dir = Path('/content/project/training_experiments')
sys.path.insert(0, str(project_dir))

# Chuyển vào thư mục training
os.chdir(project_dir)

# Import và chạy training
print("🚀 Bắt đầu training...")
print("=" * 60)

# Chạy training script
cmd = """python train_week2_lightweight.py --data_dir data/processed --epochs 50 --batch_size 32 --lr 1e-3 --use_distillation --use_qat --save_dir checkpoints/week2_colab --num_workers 2"""
os.system(cmd)

print("\n" + "=" * 60)
print("✅ Training hoàn tất!")


### Training với cấu hình tùy chỉnh


In [ ]:
# Tùy chỉnh các tham số training
EPOCHS = 50          # Số epochs
BATCH_SIZE = 32      # Batch size (tăng nếu GPU đủ mạnh)
LEARNING_RATE = 1e-3 # Learning rate
USE_QAT = True       # Quantization-Aware Training
USE_DISTILLATION = True  # Knowledge Distillation

print(f"⚙️  Cấu hình training:")
print(f"   Epochs: {EPOCHS}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Learning rate: {LEARNING_RATE}")
print(f"   QAT: {USE_QAT}")
print(f"   Distillation: {USE_DISTILLATION}")
print("\n" + "=" * 60)

# Build command
cmd = f"python train_week2_lightweight.py --data_dir data/processed --epochs {EPOCHS} --batch_size {BATCH_SIZE} --lr {LEARNING_RATE} --save_dir checkpoints/week2_colab --num_workers 2"

if USE_QAT:
    cmd += " --use_qat"
if USE_DISTILLATION:
    cmd += " --use_distillation"

# Chạy training
os.system(cmd)

print("\n" + "=" * 60)
print("✅ Training hoàn tất!")


## 8. Lưu kết quả về Google Drive


In [ ]:
import shutil
from datetime import datetime
from pathlib import Path

# Tạo thư mục lưu kết quả với timestamp
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
results_dir = Path(DRIVE_RESULTS_DIR) / f'training_{timestamp}'
results_dir.mkdir(parents=True, exist_ok=True)

# Copy checkpoints
checkpoint_dir = Path('/content/project/training_experiments/checkpoints/week2_colab')
if checkpoint_dir.exists():
    print(f"📦 Copy checkpoints...")
    shutil.copytree(checkpoint_dir, results_dir / 'checkpoints', dirs_exist_ok=True)
    print(f"✅ Đã copy checkpoints")

# Copy logs
logs_dir = checkpoint_dir / 'logs'
if logs_dir.exists():
    print(f"📊 Copy logs...")
    shutil.copytree(logs_dir, results_dir / 'logs', dirs_exist_ok=True)
    print(f"✅ Đã copy logs")

# Copy ONNX model
onnx_file = checkpoint_dir / 'mobileone_multitask.onnx'
if onnx_file.exists():
    print(f"📄 Copy ONNX model...")
    shutil.copy2(onnx_file, results_dir / 'mobileone_multitask.onnx')
    print(f"✅ Đã copy ONNX model")

print(f"\n✅ Tất cả kết quả đã được lưu tại:")
print(f"   {results_dir}")
print(f"\n📁 Bạn có thể tải về từ Google Drive sau khi training xong")


## 9. Xem kết quả training


In [ ]:
# Load và hiển thị kết quả
import torch
from pathlib import Path

checkpoint_path = Path('/content/project/training_experiments/checkpoints/week2_colab/best_model.pth')

if checkpoint_path.exists():
    checkpoint = torch.load(checkpoint_path, map_location='cpu')
    
    print("📊 Kết quả training:")
    print(f"   Epoch: {checkpoint.get('epoch', 'N/A')}")
    
    if 'val_metrics' in checkpoint:
        metrics = checkpoint['val_metrics']
        print(f"   Gender Accuracy: {metrics.get('gender_acc', 0):.4f}")
        print(f"   Age MAE: {metrics.get('age_mae', 0):.4f}")
        print(f"   Emotion Accuracy: {metrics.get('emotion_acc', 0):.4f}")
    
    if 'best_val_acc' in checkpoint:
        print(f"   Best Validation Accuracy: {checkpoint['best_val_acc']:.4f}")
else:
    print("⚠️  Chưa tìm thấy checkpoint")


## 10. TensorBoard (xem biểu đồ training)


In [ ]:
# Khởi động TensorBoard
%load_ext tensorboard
%tensorboard --logdir /content/project/training_experiments/checkpoints/week2_colab/logs


## 📝 Lưu ý quan trọng:

1. **GPU Runtime**: Luôn chọn GPU runtime để training nhanh hơn
   - Runtime → Change runtime type → Hardware accelerator → GPU

2. **Thời gian training**: 
   - Colab free: ~12 giờ/ngày, có thể bị ngắt kết nối
   - Colab Pro: ~24 giờ/ngày, ưu tiên GPU tốt hơn

3. **Lưu kết quả**: Luôn lưu kết quả về Google Drive để không bị mất khi session hết hạn

4. **Dữ liệu lớn**: Nếu dataset quá lớn, nên upload lên Google Drive trước, sau đó mount và copy

5. **Resume training**: Nếu training bị gián đoạn, có thể load checkpoint và tiếp tục:
   ```python
   checkpoint = torch.load('checkpoints/week2_colab/best_model.pth')
   model.load_state_dict(checkpoint['model_state_dict'])
   optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
   start_epoch = checkpoint['epoch'] + 1
   ```

6. **Tối ưu cho Colab**:
   - Batch size: 32-64 (tùy GPU)
   - num_workers: 2-4
   - Sử dụng mixed precision (tự động trong code)
   - Giảm epochs nếu muốn test nhanh
